<a href="https://colab.research.google.com/github/parekhravik/MLProjects/blob/master/1004_CapStoneProject_HR_ERIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Team:** 1004
**Project Name:** Employee Retantion Insight Project (ERIP)

# 1. Problem Statement

Client is looking for an solution which can predict whether an employee will leave or stay based on the past experience data collected. It will give more proactive approach to HR team in handling attritions.


**Current Practice:**
* Once an employee leaves, he or she is taken an interview with the name “exit interview” and shares reasons for leaving. The HR Department then tries and learns insights from the interview and makes changes accordingly.

* This suffers from the following problems:

> 1. This approach is that it's too haphazard. The quality of insight gained from an interview depends heavily on the skill of the interviewer.
> 2. The second problem is these insights can't be aggregated and interlaced across all employees who have left.
> 3.  The third is that it is too late by the time the proposed policy changes take effect.

Client has provided below data:

* Department Details
* Employee Personal Information (Non Sensitive)
* Employee Informations (Administrative, Work Load, Company Mutual Evaluation)

# 2. EDA


*   First Step will be to load the data and eximine.


In [8]:
!pip install mysql-connector-python

     |████████████████████████████████| 13.1MB 2.8MB/s 


In [0]:
#Imports

import mysql.connector
from mysql.connector import errorcode

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import pie, axis, show
%matplotlib inline                                                              

import warnings                                                                 # Ignore warning related to pandas_profiling
warnings.filterwarnings('ignore') 

import sqlalchemy as sqla

In [0]:
#Connect to DB and Load data

#DB Connection Details
vDBUser='student'
vDBPwd='student'
vDBHost='projects.insaid.co'
vDBSid='Capstone2'

try:
  cnx = mysql.connector.connect(user=vDBUser, password=vDBPwd,
                                host=vDBHost, database=vDBSid)
  
  pdDepartments = pd.read_sql('SELECT * FROM department_data',cnx) 
  pdEmployeeDetails = pd.read_sql('SELECT * FROM employee_details_data',cnx)  
  pdEmployeeData = pd.read_sql('SELECT * FROM employee_data',cnx)
  
except mysql.connector.Error as err:
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Something is wrong with your user name or password")
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("Database does not exist")
  else:
    print(err)
else:
  cnx.close()
  
cnx.close()

In [51]:
pdDepartments

,dept_id,dept_name,dept_head
0,D00-IT,IT,Henry Adey
1,D00-SS,Sales,Edward J Bayley
2,D00-TP,Temp,Micheal Zachrey
3,D00-ENG,Engineering,Sushant Raghunathan K
4,D00-SP,Support,Amelia Westray
5,D00-FN,Finance,Aanchal J
6,D00-PR,Procurement,Louie Viles
7,D00-AD,Admin,Evelyn Tolson
8,D00-MN,Management,Ellie Trafton
9,D00-MT,Marketing,Reuben Swann


In [47]:
pdDepartments.shape

(11, 3)

In [50]:
pdDepartments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 3 columns):
dept_id      11 non-null object
dept_name    11 non-null object
dept_head    11 non-null object
dtypes: object(3)
memory usage: 344.0+ bytes


# 2.1 Observations: department_data
1.   Our client company has total 11 departments. 
2.   This data has no missing values, each department is known by its dept_id.

Below are the columns in it:


1.   dept_id – Unique Department Code
2.   dept_name – Name of the Department
3.   dept_head – Name of the Head of the Department

In [45]:
pdEmployeeDetails.head()

,employee_id,age,gender,marital_status
0,113558,43,Male,Married
1,112256,24,Female,Unmarried
2,112586,22,Female,Unmarried
3,108071,36,Male,Married
4,116915,38,Male,Married


In [52]:
pdEmployeeDetails.shape

(14245, 4)

In [53]:
pdEmployeeDetails.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14245 entries, 0 to 14244
Data columns (total 4 columns):
employee_id       14245 non-null int64
age               14245 non-null int64
gender            14245 non-null object
marital_status    14245 non-null object
dtypes: int64(2), object(2)
memory usage: 445.2+ KB


In [54]:
pdEmployeeDetails.describe()

,employee_id,age
count,14245.000000,14245.000000
mean,112123.050544,32.889926
std,8500.457343,9.970834
min,100101.000000,22.000000
25%,105775.000000,24.000000
50%,111298.000000,29.000000
75%,116658.000000,41.000000
max,148988.000000,57.000000


# 2.1 Observations: Employee_Details
Client has total 14245 Employee Details 

Below are the columns in it:
1. employee_id – Unique ID Number for each employee
2. age – Age of the employee
3. gender – Gender of the employee
4. marital_status – Marital Status of the employee

In [46]:
pdEmployeeData.head()

,avg_monthly_hrs,department,filed_complaint,last_evaluation,n_projects,recently_promoted,salary,satisfaction,status,tenure,employee_id
0,246.0,None,NaN,0.866838,6,NaN,medium,0.134415,Left,4.0,124467
1,134.0,None,NaN,0.555718,2,NaN,low,0.511041,Left,3.0,112210
2,156.0,D00-SS,1.0,0.474082,2,NaN,medium,0.405101,Left,3.0,126150
3,256.0,D00-SP,NaN,0.961360,6,NaN,low,0.152974,Left,4.0,125346
4,146.0,D00-SS,NaN,0.507349,2,NaN,medium,0.434845,Left,3.0,113707


Let's join all 3 dataset and create a single one. Would be much better to get insights on all of the features togather.

In [0]:
pdEmployeeTemp = pd.merge(pdEmployeeData, pdEmployeeDetails, on = 'employee_id', how = 'left')

In [12]:
pdEmployeeTemp.head()

,avg_monthly_hrs,department,filed_complaint,last_evaluation,n_projects,recently_promoted,salary,satisfaction,status,tenure,employee_id,age,gender,marital_status
0,246.0,None,NaN,0.866838,6,NaN,medium,0.134415,Left,4.0,124467,42.0,Female,Married
1,134.0,None,NaN,0.555718,2,NaN,low,0.511041,Left,3.0,112210,23.0,Female,Unmarried
2,156.0,D00-SS,1.0,0.474082,2,NaN,medium,0.405101,Left,3.0,126150,24.0,Female,Unmarried
3,256.0,D00-SP,NaN,0.961360,6,NaN,low,0.152974,Left,4.0,125346,51.0,Female,Married
4,146.0,D00-SS,NaN,0.507349,2,NaN,medium,0.434845,Left,3.0,113707,23.0,Female,Unmarried


In [0]:
pdEmployeeMerged = pd.merge(pdEmployeeTemp, pdDepartments, left_on='department',right_on='dept_id', how = 'left')

In [16]:
pdEmployeeMerged.head()

,avg_monthly_hrs,department,filed_complaint,last_evaluation,n_projects,recently_promoted,salary,satisfaction,status,tenure,employee_id,age,gender,marital_status,dept_id,dept_name,dept_head
0,246.0,None,NaN,0.866838,6,NaN,medium,0.134415,Left,4.0,124467,42.0,Female,Married,NaN,NaN,NaN
1,134.0,None,NaN,0.555718,2,NaN,low,0.511041,Left,3.0,112210,23.0,Female,Unmarried,NaN,NaN,NaN
2,156.0,D00-SS,1.0,0.474082,2,NaN,medium,0.405101,Left,3.0,126150,24.0,Female,Unmarried,D00-SS,Sales,Edward J Bayley
3,256.0,D00-SP,NaN,0.961360,6,NaN,low,0.152974,Left,4.0,125346,51.0,Female,Married,D00-SP,Support,Amelia Westray
4,146.0,D00-SS,NaN,0.507349,2,NaN,medium,0.434845,Left,3.0,113707,23.0,Female,Unmarried,D00-SS,Sales,Edward J Bayley


In [17]:
#Drop duplicate department id
pdEmployeeMerged.drop(['department'], axis=1)

,avg_monthly_hrs,filed_complaint,last_evaluation,n_projects,recently_promoted,salary,satisfaction,status,tenure,employee_id,age,gender,marital_status,dept_id,dept_name,dept_head
0,246.0,NaN,0.866838,6,NaN,medium,0.134415,Left,4.0,124467,42.0,Female,Married,NaN,NaN,NaN
1,134.0,NaN,0.555718,2,NaN,low,0.511041,Left,3.0,112210,23.0,Female,Unmarried,NaN,NaN,NaN
2,156.0,1.0,0.474082,2,NaN,medium,0.405101,Left,3.0,126150,24.0,Female,Unmarried,D00-SS,Sales,Edward J Bayley
3,256.0,NaN,0.961360,6,NaN,low,0.152974,Left,4.0,125346,51.0,Female,Married,D00-SP,Support,Amelia Westray
4,146.0,NaN,0.507349,2,NaN,medium,0.434845,Left,3.0,113707,23.0,Female,Unmarried,D00-SS,Sales,Edward J Bayley
5,135.0,NaN,0.482184,2,NaN,low,0.381545,Left,3.0,121685,23.0,Female,Unmarried,D00-MT,Marketing,Reuben Swann
6,270.0,NaN,0.867087,6,NaN,low,0.172575,Left,4.0,119912,24.0,Female,Unmarried,D00-PD,Product,Darcy Staines
7,244.0,NaN,0.926197,6,NaN,medium,0.061868,Left,5.0,112868,29.0,Female,Married,D00-IT,IT,Henry Adey
8,289.0,NaN,0.929858,7,NaN,low,0.161744,Left,4.0,122314,36.0,Female,Married,D00-IT,IT,Henry Adey
9,281.0,NaN,0.907965,6,NaN,medium,0.105749,Left,4.0,122394,50.0,Female,Married,D00-SS,Sales,Edward J Bayley


In [19]:
pdEmployeeFinal.isnull().sum()

avg_monthly_hrs          0
department             707
filed_complaint      12104
last_evaluation       1487
n_projects               0
recently_promoted    13853
salary                   0
satisfaction           150
status                   0
tenure                 150
employee_id              0
age                      5
gender                   5
marital_status           5
dept_id                914
dept_name              914
dept_head              914
dtype: int64